In [2]:
# import necessary libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.metrics import roc_curve
from sklearn.metrics import roc_auc_score
%matplotlib inline

In [3]:
# load data
FIRE_DATA = pd.read_csv('POSTFIRE_MASTER_DATA.csv')

FIRE_DATA.head()

/tmp/ipykernel_50477/766390006.py:2: DtypeWarning: Columns (12,36,37) have mixed types. Specify dtype option on import or set low_memory=False.
  FIRE_DATA = pd.read_csv('POSTFIRE_MASTER_DATA.csv')


,OBJECTID,* Damage,* Street Number,* Street Name,"* Street Type (e.g. road, drive, lane, etc.)","Street Suffix (e.g. apt. 23, blding C)",* City,State,Zip Code,* CAL FIRE Unit,...,Fire Name (Secondary),APN (parcel),Assessed Improved Value (parcel),Year Built (parcel),Site Address (parcel),GLOBALID,Latitude,Longitude,x,y
0,1,No Damage,8376.0,Quail Canyon,Road,NaN,Winters,CA,NaN,LNU,...,Quail,0101090290,510000.0,1997.0,8376 QUAIL CANYON RD VACAVILLE CA 95688,e1919a06-b4c6-476d-99e5-f0b45b070de8,38.474960,-122.044465,-1.358593e+07,4.646741e+06
1,2,Affected (1-9%),8402.0,Quail Canyon,Road,NaN,Winters,CA,NaN,LNU,...,Quail,0101090270,573052.0,1980.0,8402 QUAIL CANYON RD VACAVILLE CA 95688,b090eeb6-5b18-421e-9723-af7c9144587c,38.477442,-122.043252,-1.358579e+07,4.647094e+06
2,3,No Damage,8430.0,Quail Canyon,Road,NaN,Winters,CA,NaN,LNU,...,Quail,0101090310,350151.0,2004.0,8430 QUAIL CANYON RD VACAVILLE CA 95688,268da70b-753f-46aa-8fb1-327099337395,38.479358,-122.044585,-1.358594e+07,4.647366e+06
3,4,No Damage,3838.0,Putah Creek,Road,NaN,Winters,CA,NaN,LNU,...,Quail,0103010240,134880.0,1981.0,3838 PUTAH CREEK RD WINTERS CA 95694,64d4a278-5ee9-414a-8bf4-247c5b5c60f9,38.487313,-122.015115,-1.358266e+07,4.648497e+06
4,5,No Damage,3830.0,Putah Creek,Road,NaN,Winters,CA,NaN,LNU,...,Quail,0103010220,346648.0,1980.0,3830 PUTAH CREEK RD WINTERS CA 95694,1b44b214-01fd-4f06-b764-eb42a1ec93d7,38.485636,-122.016122,-1.358277e+07,4.648259e+06


In [13]:
# explore data
FIRE_DATA.shape
FIRE_DATA.info()
FIRE_DATA.describe()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100230 entries, 0 to 100229
Data columns (total 28 columns):
 #   Column                                                        Non-Null Count   Dtype  
---  ------                                                        --------------   -----  
 0   * Damage                                                      100230 non-null  object 
 1   County                                                        100200 non-null  object 
 2   Community                                                     52892 non-null   object 
 3   Battalion                                                     6398 non-null    object 
 4   Incident Start Date                                           100230 non-null  object 
 5   Hazard Type                                                   100230 non-null  object 
 6   If Affected 1-9% - Where did fire start?                      10740 non-null   object 
 7   If Affected 1-9% - What started fire?                   

,# Units in Structure (if multi unit),# of Damaged Outbuildings < 120 SQFT,# of Non Damaged Outbuildings < 120 SQFT,Year Built (parcel),Latitude,Longitude
count,31184.000000,31085.000000,31073.00000,69812.000000,100230.000000,100230.000000
mean,0.433299,0.087566,0.12152,1672.283862,38.322953,-121.179297
std,34.608767,0.462729,0.52558,708.451814,2.019086,1.538342
min,0.000000,0.000000,0.00000,0.000000,32.592548,-123.774580
25%,0.000000,0.000000,0.00000,1944.000000,37.350926,-122.316162
50%,0.000000,0.000000,0.00000,1972.000000,38.692955,-121.600277
75%,0.000000,0.000000,0.00000,1987.000000,39.763874,-120.509278
max,6101.000000,40.000000,20.00000,2022.000000,41.991195,-116.418163


In [12]:
# clean data
FIRE_DATA = FIRE_DATA.drop(columns=['OBJECTID', '* Street Number', '* Street Name', '* Street Type (e.g. road, drive, lane, etc.)', 'Street Suffix (e.g. apt. 23, blding C)', '* City', 'State', 'Zip Code',
                                    '* CAL FIRE Unit', 'Incident Number (e.g. CAAEU 123456)', '* Incident Name', 'Fire Name (Secondary)', 'APN (parcel)', 'Assessed Improved Value (parcel)', 'Site Address (parcel)',
                                    'GLOBALID', 'x', 'y'])
